# `libzip` python bindings tutorial

First let's choose a file we will test on.

In [1]:
from pathlib import Path, PurePath
testFile = next(iter(Path("./dist/").glob("*.whl")))
testFile

PosixPath('dist/libzip-0.1.dev1+g9f16643-py3-none-any.whl')

You need to import `libzip` in order to use it. We don't put everything into the root, so import from the submodules. In order to open an archive you need `libzip.Archive.Archive` and `libzip.enums.OpenFlags`.

In [2]:
import libzip
from libzip.Archive import Archive
from libzip.enums import OpenFlags, CompressionMethod, EncryptionMethod

Since we are dealing with C resources that must be opened and closed, almost everything is a context manager.

In order to open an archive construct `Archive` class with the needed arguments.
In order to spread our stuff among different cells we'll `__enter__` the object manually.

In [3]:
a = Archive(testFile, OpenFlags.read_write | OpenFlags.check).__enter__()
a

Let's see the which files the archive contains. The archive is an iterator, so let's pass it to a constructor for a collection. Since there are pretty a lot of them, let's limit the shown by some small amount.

In [4]:
fs = list(a)
fs[:3]

[ExistingFile(<libzip.Archive.Archive object at 0x7f50e86aba40>, 0, PurePosixPath('libzip/Error.py'), <ZipFlags.enc_guess: 0>, None),
 ExistingFile(<libzip.Archive.Archive object at 0x7f50e86aba40>, 1, PurePosixPath('libzip/Stat.py'), <ZipFlags.enc_guess: 0>, None),
 ExistingFile(<libzip.Archive.Archive object at 0x7f50e86aba40>, 2, PurePosixPath('libzip/__init__.py'), <ZipFlags.enc_guess: 0>, None)]

We can get a file by its index ...

In [5]:
a[1]

ExistingFile(<libzip.Archive.Archive object at 0x7f50e86aba40>, 1, PurePosixPath('libzip/Stat.py'), <ZipFlags.enc_guess: 0>, None)

... or by name:

In [6]:
a[PurePath('libzip/Stat.py')]

ExistingFile(<libzip.Archive.Archive object at 0x7f50e86aba40>, 1, PurePosixPath('libzip/Stat.py'), <ZipFlags.enc_guess: 0>, None)

If we ask for a nonexistent file, we get an error:

In [7]:
try:
    a[100500]
except KeyError as ex:
    print(ex)

try:
    a["blah"]
except KeyError as ex:
    print(ex)

("File with this index doesn't exist in the archive", 100500)
('File not found in the archive', 'blah')


Let's examine the first file

In [8]:
f = fs[0]
display(f.stat, f.compressionMethod, f.encryptionMethod)

Stat<name=b'libzip/Error.py', index=0, originalSize=4179, compressedSize=1180, modificationTime=SubscriptableDateTime(2022, 6, 20, 18, 34, 52), crc=2528813163, compressionMethod=<CompressionMethod.deflate: 8>, encryptionMethod=<EncryptionMethod.none: 0>>

<CompressionMethod.deflate: 8>

<EncryptionMethod.none: 0>

In [9]:
display(f.modTime, f.modTime.__class__.__mro__)

SubscriptableDateTime(2022, 6, 20, 18, 34, 52)

(libzip.Stat.SubscriptableDateTime, datetime.datetime, datetime.date, object)

By setting some properties we can edit the archive. Let's change the date.

In [10]:
from datetime import datetime
f.modTime = datetime(1970, 1, 1, 0, 0, 0)
f.comment = "blah blah blah"
display(f.comment)

b'blah blah blah'

Let's recompress. When just setting a property, maximum compression is used.

In [11]:
f.compressionMethod = CompressionMethod.bzip2

To use another preset, use a method

In [12]:
f.setCompression(CompressionMethod.bzip2, 1)

For encryption you must specify the password, so we cannot encrypt by using just a prop.

In [13]:
f.setEncryption(EncryptionMethod.aes_128, "correct horse battery staple")
f.encryptionMethod

<EncryptionMethod.aes_128: 257>

To add or replace a file you need to create a new one. It is created from `Source`.

In [14]:
from libzip.Source import Source
s = Source.make(b"our new bullshit file")
f.replace(s)

To read a file enter it and use `read` method. But we cannot, we have replaced it!

In [15]:
try:
    ff = f.__enter__()
except RuntimeError as ex:
    print(ex)

Doing this op to a dirty file will result in a crash


OK, let's reset it back.

In [16]:
f.dirty = False

In [17]:
with f as ff:
    res = bytearray(f.stat.originalSize)
    ff.read(res)
    display(res[:14])

bytearray(b'import typing\n')

In [18]:
a.__exit__(None, None, None)